# Step 1: Read Watson Speech to Text timstamps

Watson Speech to Text output can return a timestamp for every word.  A words-timestamps file for a sample meeting recording is available here: [favorite-animals-short-meeting_words.json](https://raw.githubusercontent.com/spackows/CASCON-2021_Processing_video/main/sample-meeting/favorite-animals-short-meeting_words.json)

In this step, download that words-timestamps file to the notebook working directory.

In [2]:
# Download the file
import urllib.request
words_url = "https://raw.githubusercontent.com/spackows/CASCON-2021_Processing_video/main/sample-meeting/favorite-animals-short-meeting_words.json"
words_filename = "favorite-animals-short-meeting_words.json"
urllib.request.urlretrieve( words_url, words_filename )

('favorite-animals-short-meeting_words.json',
 <http.client.HTTPMessage at 0x7f1c102f5250>)

In [6]:
# View the contents of the working directory
!ls

favorite-animals-short-meeting_words.json


# Step 2: Read sentences

A file containing an array of sentences for the sample meeting recording is available here: [favorite-animals-short-meeting_sentences_arr.json](https://raw.githubusercontent.com/spackows/CASCON-2021_Processing_video/main/sample-meeting/favorite-animals-short-meeting_sentences_arr.json)

In this step, download that sentences arry file to the notebook working directory.

In [7]:
# Download the file
import urllib.request
sentences_url = "https://raw.githubusercontent.com/spackows/CASCON-2021_Processing_video/main/sample-meeting/favorite-animals-short-meeting_sentences_arr.json"
sentences_filename = "favorite-animals-short-meeting_sentences_arr.json"
urllib.request.urlretrieve( sentences_url, sentences_filename )

('favorite-animals-short-meeting_sentences_arr.json',
 <http.client.HTTPMessage at 0x7f1c102f5610>)

In [8]:
# View the contents of the working directory
!ls

favorite-animals-short-meeting_sentences_arr.json
favorite-animals-short-meeting_words.json


# Step 3: Get sentence timestamps

This step uses the timestamps in the words file to get the timestamp for the first work in each sentence.

In [66]:
# Read the words timestamps from the file
import json

with open( words_filename ) as json_file:
    words_json = json.load( json_file )
    
print( json.dumps( words_json, indent=2 )[0:700], "\n..." )

[
  [
    {
      "word": "thanks",
      "seconds": 0
    },
    {
      "word": "everybody",
      "seconds": 1
    },
    {
      "word": "for",
      "seconds": 1
    },
    {
      "word": "joining",
      "seconds": 1
    },
    {
      "word": "me",
      "seconds": 2
    },
    {
      "word": "for",
      "seconds": 2
    },
    {
      "word": "this",
      "seconds": 2
    },
    {
      "word": "short",
      "seconds": 3
    },
    {
      "word": "meeting",
      "seconds": 3
    }
  ],
  [
    {
      "word": "what",
      "seconds": 4
    },
    {
      "word": "I",
      "seconds": 5
    },
    {
      "word": "wanted",
      "seconds": 5
    },
    {
      "word": "to",
    
...


In [22]:
# Read the sentences from the file
import json

with open( sentences_filename ) as json_file:
    sentences_arr = json.load( json_file )
    
print( json.dumps( sentences_arr[0:6], indent=2 ), "\n..." )

[
  "Thanks everybody for joining me for this short meeting.",
  "What I wanted to do today was to go around the room and asked people to share what is their favorite animal and why?",
  "Okay hi my name is heather and my favorite animal is a dog and the reason that it's my favorite animal.",
  "How did a lot of people scare animal, because dogs are such loving companions who are so very loyal and I feel like they seem to know when you need them to come snuggle by you, they're very perceptive of your feelings, and they want to please you and I: don't think there are well, there are mean dogs.",
  "Well, most dogs are very lovable and only just want to please humans and so I would say.",
  "Dogs are my favorite."
] 
...


In [73]:
# Define some helper functions

import math
import re

# Watson Speech to Text returns an array of arrays, 
# so flatten that to one array
def flattenArr( chunks_arr ):
    flat_arr = []
    for words_arr in chunks_arr:
        for word_json in words_arr:
            word_json_arr = re.split( "\s+", word_json["word"] )
            for word in word_json_arr:
                flat_arr.append( { "word" : word, "seconds" : word_json["seconds"] } )
    return flat_arr

# Pretty-print the timestamp
def timestampStr( seconds ):
    num_hours = math.floor( seconds / 3600 )
    seconds_left = seconds - ( num_hours * 3600 )
    num_mins = math.floor( seconds_left / 60 )
    seconds_left = seconds_left - ( num_mins * 60 )
    hours_str = str( num_hours )
    mins_str = str( num_mins )
    sec_str = str( seconds_left )
    if num_hours < 10:
        hours_str = "0" + hours_str
    if num_mins < 10:
        mins_str = "0" + mins_str
    if seconds_left < 10:
        sec_str = "0" + sec_str
    return hours_str + ":" + mins_str + ":" + sec_str

# Very rarely, words in the words list are multiple-word terms
def handleMultiWord( words_json_flat, words_json_index, sentence_words, sentence_word_index ):
    words_arr = re.split( "\s+", words_json_flat[words_json_index]["word"] )
    for word in words_arr:
        sentence_word = sentence_words[sentence_word_index]
        sentence_word_index += 1
    return sentence_word_index;

# Given a sentence, walk along the words in that sentence 
# and walk along the corresponding words in the words-timestamps list
# to find the first word of the *next* sentence
def findNextWordIndex( words_json_index_in, sentence, words_json_flat ):
    words_json_index_next = words_json_index_in
    sentence_words = re.split( "\s+", sentence )
    sentence_word_index = 0
    while ( sentence_word_index < len( sentence_words ) ):
        sentence_word_index = handleMultiWord( words_json_flat, words_json_index_next, sentence_words, sentence_word_index )
        words_json_index_next += 1
    return words_json_index_next
    
# Step through the sentences in the sentence list, 
# finding the timestamp of the first word in that sentence
def sentenceTimestamps( words_json, sentences_arr ):
    words_json_flat = flattenArr( words_json )
    words_json_index = 0
    transcript_text = ""
    for sentence in sentences_arr:
        seconds = words_json_flat[words_json_index]["seconds"]
        transcript_text += timestampStr( seconds ) + "\n"
        transcript_text += sentence + "\n\n"
        words_json_index = findNextWordIndex( words_json_index, sentence, words_json_flat )
    return transcript_text


In [74]:
# Get the timestamps of the start of each sentence
transcripts_txt = sentenceTimestamps( words_json, sentences_arr )
print( transcripts_txt[0:750], "\n..." )

00:00:00
Thanks everybody for joining me for this short meeting.

00:00:04
What I wanted to do today was to go around the room and asked people to share what is their favorite animal and why?

00:00:17
Okay hi my name is heather and my favorite animal is a dog and the reason that it's my favorite animal.

00:00:24
How did a lot of people scare animal, because dogs are such loving companions who are so very loyal and I feel like they seem to know when you need them to come snuggle by you, they're very perceptive of your feelings, and they want to please you and I: don't think there are well, there are mean dogs.

00:00:49
Well, most dogs are very lovable and only just want to please humans and so I would say.

00:00:55
Dogs are my favorite.
 
...
